In [320]:
import nltk
import sys
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from pandas import Series,DataFrame

In [321]:
train=pd.read_csv('NewsDataset_final2.csv')

In [322]:
train=train[['Channel','topics','author','shares']]

In [323]:
train.head()

,Channel,topics,author,shares
0,Media,"amazon, Entertainment, Media, Video",Lauren Indvik,593
1,Media,"Business, Media, Twitter",Seth Fiegerman,711
2,Business,"Apple, apps, Apps and Software, Business, Mobile",Seth Fiegerman,1500
3,Entertainment,"Space, college football, Entertainment, Sports",NaN,1200
4,Tech,"apps, Apps and Software, AT&T, CES, Tech",Emily Price,505


In [324]:
lists=train['Channel']+', '+train['topics']+', '+train['author']
# here after'+' must have a space, or the Mediaamazon will be together.

In [325]:
lists

0       Media, amazon, Entertainment, Media, Video, La...
1         Media, Business, Media, Twitter, Seth Fiegerman
2       Business, Apple, apps, Apps and Software, Busi...
3                                                     NaN
4       Tech, apps, Apps and Software, AT&T, CES, Tech...
5       Tech, CES, Gadgets, Mobile, remote control, Te...
6       Gadgets, bodymedia, CES, Health & Fitness, Gad...
7       Tech, Canon, CES, Gadgets, point and shoot, Po...
8       Tech, Cars, car of the future, Include in info...
9           U.S., Politics, U.S., World, Alex Fitzpatrick
10                                                    NaN
11                                                    NaN
12      Watercooler, Alt Image Lead, Coffee, gallery, ...
13      Watercooler, jokes, Video, Videos, viral video...
14      Watercooler, Downton Abbey, gallery, Memes, tu...
15                                                    NaN
16      Watercooler, Video, Videos, viral video, Water...
17            

In [326]:
train=pd.concat([train['shares'],lists],axis=1)

In [327]:
train.rename(columns={0:'lists'},inplace=True)

In [328]:
train.head()

,shares,lists
0,593,"Media, amazon, Entertainment, Media, Video, La..."
1,711,"Media, Business, Media, Twitter, Seth Fiegerman"
2,1500,"Business, Apple, apps, Apps and Software, Busi..."
3,1200,NaN
4,505,"Tech, apps, Apps and Software, AT&T, CES, Tech..."


In [329]:
train['shares'].describe()

count      3996.000000
mean       3237.111361
std       11786.715153
min         217.000000
25%         998.750000
50%        1600.000000
75%        2800.000000
max      617900.000000
Name: shares, dtype: float64

In [330]:
# delete all NaN, binned the shares
train=train.dropna()

def binn(data):
    
    data.loc[data['shares'] < 998, 'shares'] = 1
    data.loc[(data['shares'] < 1600) & (data['shares'] >= 998), 'shares'] = 2
    data.loc[(data['shares'] < 2800) & (data['shares'] >=1600), 'shares'] = 3
    data.loc[data['shares'] >= 2800, 'shares'] = 4
    train.loc[train['shares']==1, 'shares'] = 'low'
    train.loc[train['shares']==2, 'shares'] = 'medium'
    train.loc[train['shares']==3, 'shares'] = 'high'
    train.loc[train['shares']==4, 'shares'] = 'best'
    
    return data
train=binn(train)
train.head()

,shares,lists
0,low,"Media, amazon, Entertainment, Media, Video, La..."
1,low,"Media, Business, Media, Twitter, Seth Fiegerman"
2,medium,"Business, Apple, apps, Apps and Software, Busi..."
4,low,"Tech, apps, Apps and Software, AT&T, CES, Tech..."
5,low,"Tech, CES, Gadgets, Mobile, remote control, Te..."


In [331]:
# now, need get off all 1.punctuation, comma .. and all 2. stopwords.
#lets take one as example
train.loc[train['shares']==1, 'shares'] = 'asd'
train['shares']

0          low
1          low
2       medium
4          low
5          low
6          low
7          low
8         best
9          low
12         low
13        best
14         low
16        best
18        best
19        best
20        best
21         low
22         low
23      medium
24         low
25         low
26      medium
27        high
28         low
29         low
31        best
32         low
33         low
34        best
35         low
         ...  
3964      best
3965      best
3966       low
3967    medium
3968    medium
3969       low
3970      best
3971      high
3972      high
3973       low
3974      best
3975      best
3976    medium
3977      best
3978    medium
3979      high
3980       low
3982      best
3983       low
3984      high
3985      high
3987      best
3988    medium
3989    medium
3990    medium
3991       low
3992       low
3993    medium
3994    medium
3995    medium
Name: shares, Length: 3436, dtype: object

In [332]:
message=train['lists'][0]
message

'Media, amazon, Entertainment, Media, Video, Lauren Indvik'

In [333]:
# 1. get off punctuation, such as comma
import string
nonpunc = [char for char in message if char not in string.punctuation]
# here use message rather than message.split(), caz need check every letter to find comma
print(nonpunc,end='')

['M', 'e', 'd', 'i', 'a', ' ', 'a', 'm', 'a', 'z', 'o', 'n', ' ', 'E', 'n', 't', 'e', 'r', 't', 'a', 'i', 'n', 'm', 'e', 'n', 't', ' ', 'M', 'e', 'd', 'i', 'a', ' ', 'V', 'i', 'd', 'e', 'o', ' ', 'L', 'a', 'u', 'r', 'e', 'n', ' ', 'I', 'n', 'd', 'v', 'i', 'k']

In [334]:
nonpunc=''.join(nonpunc)
nonpunc  # join them

'Media amazon Entertainment Media Video Lauren Indvik'

In [335]:
# 2. get off all stopwords.
import nltk
nltk.download('stopwords')# here must use the split or 'word' will be treated as letter not word
cleann = [word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to C:\Users\ZHAOFENG
[nltk_data]     SHANG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [336]:
cleann
## in the front, after'+' must have a space, or the Mediaamazon will be together.

['Media', 'amazon', 'Entertainment', 'Media', 'Video', 'Lauren', 'Indvik']

In [337]:
# after example and dataset cleaning, need seperate the dataset into train and test part.
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

In [339]:
# then, we need combine two steps together into a function

def text_process(data):
    
    nonpunc = [char for char in data if char not in string.punctuation] # out punctuation
    nonpunc=''.join(nonpunc)
    
    cleann = [words for words in nonpunc if words.lower() not in stopwords.words('english')]
    return cleann


In [371]:
x,y=train.drop(['shares'],axis=1),train['shares']


In [340]:
bow_CountVectorizer = CountVectorizer(analyzer=text_process)
transformer=TfidfTransformer()

In [364]:
tfidf=transformer.fit_transform(bow_CountVectorizer.fit_transform(x['lists']))

In [365]:
tfidf

<3436x51 sparse matrix of type '<class 'numpy.float64'>'
	with 49222 stored elements in Compressed Sparse Row format>

In [366]:
x_train, x_test, y_train, y_test = train_test_split( tfidf, y, test_size = 0.3, random_state = 100)

In [367]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(x_train,y_train)

In [368]:

predictions = model.predict(x_test)

In [370]:
#rint("Accuracy is", accuracy_score(predictions,y_test))